In [1]:
import numpy as np
from osgeo import gdal

In [2]:
#loading aspect, called flowraster
flowraster_file = '/Volumes/NO NAME/masterarbete/SuPy/DataJuliasExercise/flowraster.tif'
flowraster = gdal.Open(flowraster_file).ReadAsArray().astype(np.float)
flowraster.shape[0] #row in raster
flowraster.shape[1] #column in raster

# Loading landcover, called lc. loading the file clipped to my polygon
lcGrid_file = '/Volumes/NO NAME/masterarbete/SuPy/DataJuliasExercise/landcover.tif'
lcGrid = gdal.Open(lcGrid_file).ReadAsArray().astype(np.float)

ipykernel_launcher:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
ipykernel_launcher:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [3]:
#create empty matrix
col = 9
row = 7 
matrix = np.zeros((row, col))
index = 0

In [4]:
#Gradantal för varje kompassriktning
n = 0. 
ne = 45.
e = 90.
se = 135.
s = 180.
sw = 225.
w = 270.
nw = 315.

In [5]:
ToRunOffGrass = 0.
ToSoilStoreGrass = 0.4

ToRunOffPaved = 0.9
ToSoilStorePaved = 0.

ToRunOffBuildings = 0.9
ToSoilStoreBuildings = 0.

ToRunOffEvergreen = 0.
ToSoilStoreEvergreen = 0.4

ToRunOffDecid = 0.
ToSoilStoreDecid = 0.4

ToRunOffBareSoil = 0.
ToSoilStoreBareSoil = 0.4

ToRunOffWater = 0.
ToSoilStoreWater = 1.

#tillagt
ToBldgsPaved = 0.
ToGrassPaved = 0.3

matrix[0,7] = ToRunOffPaved
matrix[0,8] = ToSoilStorePaved

matrix[1,7] = ToRunOffBuildings
matrix[1,8] = ToSoilStoreBuildings

matrix[2,7] = ToRunOffEvergreen
matrix[2,8] = ToSoilStoreEvergreen

matrix[3,7] = ToRunOffDecid
matrix[3,8] = ToSoilStoreDecid

matrix[4,7] = ToRunOffGrass
matrix[4,8] = ToSoilStoreGrass

matrix[5,7] = ToRunOffBareSoil
matrix[5,8] = ToSoilStoreBareSoil

matrix[6,7] = ToRunOffWater
matrix[6,8] = ToSoilStoreWater

#försök till att lägga värde som restricted, eller alltså att det inte ska få gå från paved till build
matrix[0, 1] = ToBldgsPaved
matrix[0, 4] = ToGrassPaved


In [6]:
for i in np.arange(1, flowraster.shape[0] - 1): 
    for j in np.arange(1, flowraster.shape[1] - 1):

        lcTypeMid = lcGrid[i, j] 
        nFrac = 0.
        neFrac = 0. 
        eFrac = 0. 
        seFrac = 0.
        sFrac = 0.
        swFrac = 0.
        wFrac = 0.
        nwFrac = 0.
        dirflow = flowraster[i, j] 
        if dirflow == 0.:
            dirflow = 360.
        if (dirflow >= n) and (dirflow < ne): 
            flowfrac = (ne -dirflow) / 45.
            nFrac = flowfrac
            neFrac = 1. - flowfrac
        
        if (dirflow >= ne) and (dirflow < e):
            flowfrac = (e - dirflow) / 45.
            neFrac = flowfrac
            eFrac = 1. - flowfrac

        if (dirflow >=e) and (dirflow < se):
            flowfrac = (se - dirflow) / 45.
            eFrac = flowfrac
            seFrac = 1. - flowfrac
            
        if (dirflow >=se) and (dirflow < s):
            flowfrac = (s - dirflow) / 45.
            seFrac = flowfrac
            sFrac = 1. - flowfrac 
            
        if (dirflow >= s) and (dirflow < sw):
            flowfrac = (sw - dirflow) / 45.
            sFrac = flowfrac
            swFrac = 1. - flowfrac
            
        if (dirflow >= sw) and (dirflow < w):
            flowfrac = (w - dirflow) / 45.
            swFrac = flowfrac
            wFrac = 1. - flowfrac
            
        if (dirflow >= w) and (dirflow < nw):
            flowfrac = (nw - dirflow) / 45. 
            wFrac = flowfrac
            nwFrac = 1. - flowfrac
            
        if (dirflow >= nw):
            flowfrac = (360 - dirflow) / 45.
            nwFrac = flowfrac
            nFrac = 1. - flowfrac
        
        
        
        #for k in np.arange(0, 8):
        matrixtemp = np.zeros((7, 9))     
        if nFrac > 0:
            lcType = lcGrid[i - 1, j] #n
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = nFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]
        if neFrac > 0:
            lcType = lcGrid[i - 1, j + 1] #ne
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = neFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]
        if eFrac > 0:
            lcType = lcGrid[i, j + 1] #e
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = eFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]
        if seFrac > 0:
            lcType = lcGrid[i + 1, j + 1] #se
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = seFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]
        if sFrac > 0:
            lcType = lcGrid[i + 1, j] #s
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = sFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]      
        if swFrac > 0:
            lcType = lcGrid[i + 1, j - 1] #sw
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = swFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]
        if wFrac > 0:
            lcType = lcGrid[i, j - 1] #w
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = wFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]
        if nwFrac > 0:
            lcType = lcGrid[i - 1, j - 1] #nw
            if not lcType == lcTypeMid:
                matrixtemp[int(lcType - 1), int(lcTypeMid - 1)] = nwFrac + matrixtemp[int(lcType - 1), int(lcTypeMid - 1)]
        matrix = matrix + matrixtemp
        index += 1
        test = 4

In [7]:
matrixfinal = np.copy(matrix)
for o in range(0, 7): #rad
    sumtemp = np.sum(matrixfinal[o,:6])
    for p in range(0, 7): #kolumn
        matrixfinal[o, p] = (matrixfinal[o, p] / sumtemp) * (1 - (matrix[o, 7] + matrix[o, 8]))

ipykernel_launcher:5: RuntimeWarning: invalid value encountered in double_scalars


In [8]:
test = 5

header = 'Paved Buildings Evergreen Deciduous Grass Baresoil Water ToRunOff ToSoilStore'
numformat = '%8.2f '* 9
np.savetxt('/Volumes/NO NAME/masterarbete/SuPy/DataJuliasExercise/VilleLcAspectTEST.txt', matrixfinal, fmt=numformat, header=header, delimiter=' , ')